# 特征：
- 使用Distance（空值填1）
- Discount_rate做one-hot
- 每个用户：优惠券使用次数
- 领券是星期几
- 是否小于15天消费
- 商家优惠券被消费次数
- 用户当天使用优惠券
- 只在一个商铺领取过优惠劵并且有消费记录
- 用户使用该商家的优惠劵的平均时间
- 用户消费的平均距离
- 用户使用某种类型的优惠劵数目/用户领取优惠劵总数
- 用户领取某种类型的优惠劵数目/用户领取优惠劵总数
- 用户在某个商铺的优惠券领取数
- 用户在某个商铺的优惠券消费数
- 用户只领取过一次优惠劵但未消费
- 用户领取优惠劵并且全部消费
# 算法：逻辑回归

In [1]:
import pandas as pd
import numpy as np

off_train = pd.read_csv('E:/tianchi/ccf_offline_stage1_train.csv')
off_test = pd.read_csv('E:/tianchi/ccf_offline_stage1_test_revised.csv')
off_train = off_train.dropna(subset = ['Coupon_id'])

# 加标签

In [2]:
off_train['result'] = off_train[['Coupon_id', 'Date']].apply(lambda x: 1 if pd.notnull(x[0]) and pd.notnull(x[1]) else 0, axis = 1)

# 加领券是星期几

In [3]:
#对训练集取星期数
off_train["Week"] = off_train["Date_received"]
off_train.Week = off_train.Week.astype("int")
off_train.Week = off_train.Week.astype("str")
off_train["Week"] = pd.to_datetime(off_train["Week"]).apply(lambda x : x.weekday())

#对测试集取星期数
off_test["Week"] = off_test["Date_received"]
off_test.Week = off_test.Week.astype("int")
off_test.Week = off_test.Week.astype("str")
off_test["Week"] = pd.to_datetime(off_test["Week"]).apply(lambda x : x.weekday())

# 加是否小于15天消费

In [4]:
import datetime

#判断使用优惠券日期-领取优惠券日期是否小于15天，小于取1，大于取0 
def days(z):
    if str(z['Date_received']) != 'nan' and str(z['Date']) != 'nan':
        days = (datetime.datetime.strptime(str(int(z['Date'])), "%Y%m%d") - datetime.datetime.strptime(str(int(z['Date_received'])), "%Y%m%d"))
        if days.days > 15:
            return 0
        else:
            return 1
    else:
        return 0

#将判断后的结果赋值给训练集    
off_train['less_15'] = off_train.apply(days, axis = 1)

#用户和优惠券id是多对多的关系，所以按用户对他消费小于15天的数做了一个累加，去除掉训练集中的这列特征，方便做处理后特征的连接
train = off_train[['User_id', 'less_15']]
train = train.drop_duplicates(subset = ['User_id'])
a = train.less_15.groupby(train['User_id']).sum()
train = a.reset_index()
off_train = off_train.drop(columns = 'less_15')

#对测试集和测试集按用户增加这列特征
off_train = pd.merge(off_train, train, on = 'User_id', how = 'left')
off_test = pd.merge(off_test, train, on = 'User_id', how = 'left')

#测试集中有训练集中没有出现的用户，这类用户的这个特征默认填了空，把空值填0
off_test.less_15 = off_test.less_15.fillna(0)

# 加用户消费次数

In [5]:
#统计用户消费次数
User_Coupon_notna = off_train[['User_id', 'Date']].groupby('User_id').count()
User_Coupon_notna = User_Coupon_notna.reset_index()
User_Coupon_notna.rename(columns = {'Coupon_id' : 'Coupon_id_count', 'Date' : 'Consume_count'}, inplace=True)

#将得到的特征返回到训练集与测试集中，测试集中有训练集中没有出现的用户，这类用户的这个特征默认填了空，把空值填0
off_train = pd.merge(off_train, User_Coupon_notna[['User_id', 'Consume_count']], on = ['User_id'], how = 'left')
off_test = pd.merge(off_test, User_Coupon_notna[['User_id', 'Consume_count']], on = 'User_id', how = 'left')
off_test = off_test.fillna(value = {'Consume_count' : 0})

# Distinct填1

In [6]:
off_train.Distance = off_train.Distance.fillna(1)
off_test.Distance = off_test.Distance.fillna(1)

# 加商家优惠券被消费次数

In [7]:
#统计商家优惠券消费次数
Merchant_Coupon_notna = off_train['Date'].groupby(off_train['Merchant_id']).count()
Merchant_Coupon_notna = Merchant_Coupon_notna.reset_index()
Merchant_Coupon_notna.rename(columns = {'Date' : 'Coupon_have_userd_counts'}, inplace = True)

#将得到的特征返回到训练集与测试集中，测试集中有训练集中没有出现的商家，这类商家的这个特征默认填了空，把空值填0
off_train = pd.merge(off_train, Merchant_Coupon_notna, on = ['Merchant_id'], how = 'left')
off_test = pd.merge(off_test, Merchant_Coupon_notna ,on = ['Merchant_id'], how='left')
off_test = off_test.fillna(value = {'Coupon_have_userd_counts' : 0})

# 加用户当天使用优惠券

In [11]:
#找出有优惠券消费的数据
date_nan_drop = off_train.dropna(subset=['Date'])
date_nan_drop = date_nan_drop.reset_index()

#领取日期等于消费日期，说明是当天消费
import datetime
def Use(z):
    if z.Date_received == z.Date:
        return 1
    else:
        return np.NaN
date_nan_drop['Use_today'] = date_nan_drop.apply(Use, axis=1)
#一个用户可能有多个当天消费行为，将这部分去重
a = date_nan_drop[['User_id', 'Use_today']]
a = a.drop_duplicates(subset=['User_id'])
#将结果返回
off_train = pd.merge(off_train, a, on = 'User_id', how = 'left')
off_test = pd.merge(off_test, a, on = 'User_id', how = 'left')
#空值填0
off_train = off_train.fillna({'Use_today':0})
off_test = off_test.fillna({'Use_today':0})

# 加只在一个商铺领取过优惠劵并且有消费记录

In [12]:
#找出有优惠券消费的数据
date_nan_drop = off_train.dropna(subset=['Date'])
date_nan_drop = date_nan_drop.reset_index()
#找出用户在几个商家消费过，只在1个置1，其余置0
Consume_oneMerchant = date_nan_drop['Merchant_id'].groupby(date_nan_drop['User_id'])
a = Consume_oneMerchant.count()
a = a.reset_index()
a = a.rename(columns={'Merchant_id':'Use_One_Merchant'})
a['Use_One_Merchant'] = a.Use_One_Merchant.apply(lambda x:1 if x == 1 else 0)
#将结果返回
off_train = pd.merge(off_train, a, on = 'User_id', how = 'left')
off_test = pd.merge(off_test, a, on = 'User_id', how = 'left')
#空值填0
off_train = off_train.fillna({'Use_One_Merchant':0})
off_test = off_test.fillna({'Use_One_Merchant':0})

# 加用户使用该商家的优惠劵的平均时间

In [13]:
import datetime

#返回领取到消费天数
def days(z):
    if str(z['Date_received']) != 'nan' and str(z['Date']) != 'nan':
        days = (datetime.datetime.strptime(str(int(z['Date'])), "%Y%m%d") - datetime.datetime.strptime(str(int(z['Date_received'])), "%Y%m%d"))
        return days.days
    else:
        return np.NaN
    
off_train['days'] =  off_train.apply(days, axis=1)
#找出有消费的数据
use_Coupon = off_train.dropna(subset=['days'])
#按用户计算天数均值
a = use_Coupon['days'].groupby(use_Coupon['User_id']).mean()
a = a.reset_index()
a = a.rename(columns={'days':'Use_mean_time'})
#将结果返回测试集与训练集
off_train = pd.merge(off_train, a, on = 'User_id', how = 'left')
off_test = pd.merge(off_test, a, on = 'User_id', how = 'left')
#空值填0
off_train = off_train.fillna({'Use_mean_time':0})
off_test = off_test.fillna({'Use_mean_time':0})

# 加用户消费的平均距离

In [14]:
#找出有消费的数据
use_Coupon = off_train.dropna(subset=['days'])
#按用户计算距离均值
a = use_Coupon['Distance'].groupby(use_Coupon['User_id']).mean()
a = a.reset_index()
a = a.rename(columns={'Distance':'Use_mean_Distance'})
#将结果返回训练集与测试集
off_train = pd.merge(off_train, a, on = 'User_id', how = 'left')
off_test = pd.merge(off_test, a, on = 'User_id', how = 'left')
#空值填0
off_train = off_train.fillna({'Use_mean_Distance':0})
off_test = off_test.fillna({'Use_mean_Distance':0})

# 加用户使用某种类型的优惠劵数目/用户领取优惠劵总数

In [15]:
#找出有消费的数据
use_Coupon = off_train.dropna(subset=['days'])
#找出用户使用某种类型的优惠券数目
a = use_Coupon[['User_id', 'Discount_rate']].groupby([use_Coupon['User_id'], use_Coupon['Discount_rate']]).count()
a = a.rename(columns={'User_id':'Coupon_use'})
a = a.drop(columns=['Discount_rate'])
a = a.reset_index()
#找不用户领取优惠券总数
b = off_train['Date_received'].groupby(off_train['User_id']).count()
b = b.reset_index()
b = b.rename(columns={'Date_received':'Coupon_recieved'})
#将他们相除
c = pd.merge(a, b, on = 'User_id', how = 'left')
def chufa(z):
    x = z.Coupon_use / z.Coupon_recieved
    return x
c['Coupon_use/Coupon_sum'] = c.apply(chufa, axis=1)
#将结果返回训练集与测试集
off_train = pd.merge(off_train, c[['User_id', 'Discount_rate', 'Coupon_use/Coupon_sum']], on = ['User_id', 'Discount_rate'], how = 'left')
off_test = pd.merge(off_test, c[['User_id', 'Discount_rate', 'Coupon_use/Coupon_sum']], on = ['User_id', 'Discount_rate'], how = 'left')
#空值填0
off_train = off_train.fillna({'Coupon_use/Coupon_sum':0})
off_test = off_test.fillna({'Coupon_use/Coupon_sum':0})

# 加用户领取某种类型的优惠劵数目/用户领取优惠劵总数

In [16]:
#找出用户领取某种类型的优惠劵数目
a = off_train[['User_id', 'Discount_rate']].groupby([off_train['User_id'], off_train['Discount_rate']]).count()
a = a.rename(columns={'User_id':'Coupon_use'})
a = a.drop(columns=['Discount_rate'])
a = a.reset_index()
#找出用户领取优惠劵总数
b = off_train['Date_received'].groupby(off_train['User_id']).count()
b = b.reset_index()
b = b.rename(columns={'Date_received':'Coupon_recieved'})
#将他们相除
c = pd.merge(a, b, on = 'User_id', how = 'left')
def chufa(z):
    x = z.Coupon_use / z.Coupon_recieved
    return x
c['Coupon/Coupon_sum'] = c.apply(chufa, axis=1)
#将结果返回训练集与测试集
off_train = pd.merge(off_train, c[['User_id', 'Discount_rate', 'Coupon/Coupon_sum']], on = ['User_id', 'Discount_rate'], how = 'left')
off_test = pd.merge(off_test, c[['User_id', 'Discount_rate', 'Coupon/Coupon_sum']], on = ['User_id', 'Discount_rate'], how = 'left')
#空值填0
off_train = off_train.fillna({'Coupon/Coupon_sum':0})
off_test = off_test.fillna({'Coupon/Coupon_sum':0})

# Discount_rate独热

In [17]:
#对训练集和测试集中的Discount_rate进行独热编码
off_train = pd.get_dummies(off_train, prefix=['Discount_rate'])
off_test = pd.get_dummies(off_test, prefix=['Discount_rate'])

#训练集中出现了4个测试集中没有出现的码段，进行填充
m = list(set(off_train.columns.tolist()) - set(off_test.columns.tolist()))
for i in m:
    off_test[i] = 0

#测试集中有一个训练集中没有的码段，进行填充    
off_train['Discount_rate_500:30'] = 0

# 加用户在某个商铺的优惠券领取数

In [18]:
#找出用户在某个商铺的优惠券领取数
a = off_train['Date_received'].groupby([off_train['User_id'], off_train['Merchant_id']]).count()
a = a.reset_index()
a = a.rename(columns={'Date_received':'User_Merchant_Coupon'})
#将结果返回训练集与测试集
off_train = pd.merge(off_train, a, on = ['User_id', 'Merchant_id'], how = 'left')
off_test = pd.merge(off_test, a, on = ['User_id', 'Merchant_id'], how = 'left')
#空值填0
off_train = off_train.fillna({'User_Merchant_Coupon':0})
off_test = off_test.fillna({'User_Merchant_Coupon':0})

# 加用户在某个商铺的优惠券消费数

In [19]:
#找出用户在某个商铺的优惠券消费数
a = off_train['Date'].groupby([off_train['User_id'], off_train['Merchant_id']]).count()
a = a.reset_index()
a = a.rename(columns={'Date':'User_Merchant_Coupon_used'})
#将结果返回训练集与测试集
off_train = pd.merge(off_train, a, on = ['User_id', 'Merchant_id'], how = 'left')
off_test = pd.merge(off_test, a, on = ['User_id', 'Merchant_id'], how = 'left')
#空值填0
off_train = off_train.fillna({'User_Merchant_Coupon_used':0})
off_test = off_test.fillna({'User_Merchant_Coupon_used':0})

# 用户只领取过一次优惠劵但未消费

In [20]:
user_coupon = off_train[['User_id', 'Coupon_id']]
# 利用User_id和Coupon_id两列对用户进性分组，并且计算每组的数目，并进行序号重置
user_coupon_count = user_coupon.groupby(by=['User_id']).size().reset_index()
# 重命名
user_coupon_count.rename(columns={0:'Coupon_size'}, inplace=True)
# 将筛选出来需要merge的列和原始的数据进行merge Coupon_size代表用户领取的优惠劵数目
off_train = off_train.merge(user_coupon_count, on='User_id', how='inner')

user_coupon = off_train[['User_id', 'result']]
# 利用User_id和result两列对用户进性分组，并且计算每组的数目（这里相当于求和），并进行序号重置
user_coupon_used_count = user_coupon.groupby(by=['User_id']).sum().reset_index()
# 重命名 Consume_count用户使用了的优惠劵数目
user_coupon_used_count.rename(columns={'result':'Conupon_have_used_count'}, inplace=True)
# 将筛选出来需要merge的列和原始的数据进行merge
off_train = off_train.merge(user_coupon_used_count, on='User_id', how='inner')

# 为训练集添加Coupon_all_consume特征，标记那些领取优惠劵数目等于使用优惠劵数目
off_train['Coupon_all_consume'] = off_train[['Conupon_have_used_count','Coupon_size']].apply(lambda x : 1 if x[0] == x[1] else 0, axis = 1)
# 去掉产生的中间列
off_train = off_train.drop(columns=['Coupon_size', 'Conupon_have_used_count'])
# 提取需要的列好merge到测试集中
off_train_coupon_all_consume = off_train[['User_id', 'Coupon_all_consume']]
off_train_coupon_all_consume = off_train_coupon_all_consume.drop_duplicates(['User_id', 'Coupon_all_consume'])
#为测试集添加Coupon_all_consume特征
off_test = off_test.merge(off_train_coupon_all_consume, on='User_id', how='left')
# 填充merge以后的空值为0
off_test = off_test.fillna({'Coupon_all_consume':0})

# 用户领取优惠劵并且全部消费

In [21]:
user_coupon = off_train[['User_id', 'Coupon_id','result']]
# 利用User_id和result两列对用户进性分组，并且计算每组的数目，并进行序号重置
user_coupon_count = user_coupon.groupby(by=['User_id','result']).size().reset_index()
# 重命名
user_coupon_count.rename(columns={0:'Coupon_size'}, inplace=True)
# 筛选出领取次数为1并且未消费的
user_coupon_count_1 = user_coupon_count[(user_coupon_count.Coupon_size == 1) & (user_coupon_count.result == 0)]
# 去掉不需要merge的这一列，即result
user_coupon_count_1 = user_coupon_count_1.drop(columns=['result'])

# 训练集进行添加One_coupon_no_consume特征
# 将筛选出来需要merge的列和原始的数据进行merge
off_train = off_train.merge(user_coupon_count_1, on='User_id', how='left')
#再次重命名
off_train.rename(columns={'Coupon_size':'One_coupon_no_consume'}, inplace=True)
# 填充merge以后的空值项为 0
off_train = off_train.fillna({'One_coupon_no_consume':0})

# 测试集添加One_coupon_no_consume特征
off_test = off_test.merge(user_coupon_count_1, on='User_id', how='left')
off_test.rename(columns={'Coupon_size':'One_coupon_no_consume'}, inplace=True)
off_test = off_test.fillna({'One_coupon_no_consume':0})

In [22]:
off_train = off_train.fillna(0)
off_test = off_test.fillna(0)

In [23]:
#训练数据、训练标签、测试数据的设置
x_train = off_train.drop(columns = ['Date_received', 'User_id', 'Merchant_id', 'Coupon_id', 'Date', 'result', 'days'])
y_train = off_train.result
x_test = off_test.drop(columns = ['Date_received', 'User_id', 'Merchant_id', 'Coupon_id'])

#将训练数据和测试数据的列一一对应
x_test = x_test[x_train.columns.tolist()]

from sklearn.linear_model import LogisticRegression

#调用逻辑回归模型，并进行训练
clf = LogisticRegression()
clf.fit(x_train, y_train)

#进行预测，并将需要保存的部分写入save
predict = clf.predict_proba(x_test)
save = off_test[['User_id', 'Coupon_id', 'Date_received']]
save.insert(3, 'probability', predict[:,1], True)

#输出结果为csv
save.to_csv('E:/tianchi/20180726/all_feature.csv', index = None)